## חקר אשכולות K-Means באמצעות R ועקרונות נתונים מסודרים

### [**שאלון לפני השיעור**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

בשיעור זה תלמדו כיצד ליצור אשכולות באמצעות חבילת Tidymodels וחבילות נוספות באקוסיסטם של R (נקרא להן חברים 🧑‍🤝‍🧑), תוך שימוש במאגר הנתונים של מוזיקה ניגרית שייבאתם קודם לכן. נסקור את היסודות של K-Means לאשכולות. זכרו, כפי שלמדתם בשיעור הקודם, ישנן דרכים רבות לעבוד עם אשכולות, והשיטה שתבחרו תלויה בנתונים שלכם. ננסה את K-Means מכיוון שזו טכניקת האשכולות הנפוצה ביותר. בואו נתחיל!

מונחים שתלמדו עליהם:

- ניקוד סילואט (Silhouette scoring)

- שיטת המרפק (Elbow method)

- אינרציה (Inertia)

- שונות (Variance)

### **מבוא**

[אשכולות K-Means](https://wikipedia.org/wiki/K-means_clustering) היא שיטה שמקורה בתחום עיבוד האותות. היא משמשת לחלוקה וקיבוץ של נתונים ל-`k אשכולות` בהתבסס על דמיון בתכונותיהם.

ניתן להמחיש את האשכולות כ-[דיאגרמות וורונוי](https://wikipedia.org/wiki/Voronoi_diagram), הכוללות נקודה (או 'זרע') והאזור המתאים לה.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>אינפוגרפיקה מאת ג'ן לופר</figcaption>

תהליך אשכולות K-Means כולל את השלבים הבאים:

1. מדען הנתונים מתחיל בקביעת מספר האשכולות הרצוי ליצירה.

2. לאחר מכן, האלגוריתם בוחר באופן אקראי K תצפיות ממאגר הנתונים שישמשו כמרכזים הראשוניים של האשכולות (כלומר, צנטרואידים).

3. לאחר מכן, כל אחת מהתצפיות הנותרות מוקצת לצנטרואיד הקרוב ביותר.

4. לאחר מכן, מחושב הממוצע החדש של כל אשכול, והצנטרואיד מועבר לממוצע.

5. כעת, לאחר שחושבו מחדש המרכזים, נבדקת שוב כל תצפית כדי לראות אם היא קרובה יותר לאשכול אחר. כל האובייקטים מוקצים מחדש תוך שימוש בממוצעי האשכולות המעודכנים. שלבי הקצאת האשכולות ועדכון הצנטרואידים חוזרים על עצמם באופן איטרטיבי עד שהקצאות האשכולות מפסיקות להשתנות (כלומר, כאשר מושגת התכנסות). בדרך כלל, האלגוריתם מסתיים כאשר כל איטרציה חדשה מביאה לתנועה זניחה של הצנטרואידים והאשכולות הופכים לסטטיים.

<div>

> שימו לב שבשל האקראיות בבחירת k התצפיות הראשוניות כצנטרואידים ההתחלתיים, אנו עשויים לקבל תוצאות מעט שונות בכל פעם שניישם את התהליך. מסיבה זו, רוב האלגוריתמים משתמשים בכמה *התחלות אקראיות* ובוחרים את האיטרציה עם ה-WCSS הנמוך ביותר. לכן, מומלץ בחום להריץ תמיד את K-Means עם כמה ערכים של *nstart* כדי להימנע מ-*מינימום מקומי בלתי רצוי.*

</div>

אנימציה קצרה זו, המשתמשת ב-[איורים](https://github.com/allisonhorst/stats-illustrations) של אליסון הורסט, מסבירה את תהליך האשכולות:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>איור מאת @allison_horst</figcaption>

שאלה בסיסית שעולה באשכולות היא: איך יודעים לכמה אשכולות לחלק את הנתונים? חיסרון אחד של שימוש ב-K-Means הוא שתצטרכו לקבוע את `k`, כלומר את מספר ה-`צנטרואידים`. למרבה המזל, `שיטת המרפק` עוזרת להעריך ערך התחלתי טוב עבור `k`. תנסו זאת בעוד רגע.

### 

**דרישות מוקדמות**

נמשיך בדיוק מהמקום שבו עצרנו ב-[שיעור הקודם](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), שבו ניתחנו את מאגר הנתונים, יצרנו המון ויזואליזציות וסיננו את הנתונים לתצפיות מעניינות. ודאו שבדקתם אותו!

נזדקק לכמה חבילות כדי להשלים את המודול הזה. ניתן להתקין אותן כך: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

לחילופין, הסקריפט הבא בודק אם יש לכם את החבילות הנדרשות להשלמת המודול ומתקין אותן עבורכם במקרה שחסרות:


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


בואו נתחיל!

## 1. ריקוד עם נתונים: צמצום ל-3 הז'אנרים המוזיקליים הפופולריים ביותר

זהו סיכום של מה שעשינו בשיעור הקודם. בואו נחתוך וננתח קצת נתונים!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 זה הלך טוב!

## 2. חקירת נתונים נוספת.

כמה נקיים הנתונים האלה? בואו נבדוק חריגות באמצעות תרשימי קופסה. נתמקד בעמודות מספריות עם פחות חריגות (למרות שאתם יכולים לנקות את החריגות). תרשימי קופסה יכולים להראות את טווח הנתונים ויעזרו לבחור אילו עמודות להשתמש. שימו לב, תרשימי קופסה אינם מציגים שונות, שהיא מרכיב חשוב בנתונים שניתן לקבץ בצורה טובה. אנא ראו [דיון זה](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) לקריאה נוספת.

[תרשימי קופסה](https://en.wikipedia.org/wiki/Box_plot) משמשים לתיאור גרפי של התפלגות נתונים `מספריים`, אז בואו נתחיל ב*בחירת* כל העמודות המספריות לצד ז'אנרי המוזיקה הפופולריים.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


ראו כיצד הפונקציה `where` עוזרת בבחירה בצורה פשוטה 💁? תוכלו לחקור פונקציות נוספות [כאן](https://tidyselect.r-lib.org/).

מכיוון שניצור תרשים תיבות עבור כל מאפיין מספרי ורוצים להימנע משימוש בלולאות, נשנה את מבנה הנתונים שלנו לפורמט *ארוך* יותר, שיאפשר לנו לנצל את `facets` - תת-תרשימים שכל אחד מהם מציג תת-קבוצה של הנתונים.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


קל-ג'י-ג'י!

עכשיו אנחנו יכולים לראות שהנתונים מעט רועשים: על ידי התבוננות בכל עמודה כגרף קופסה, ניתן להבחין בערכים חריגים. אפשר לעבור על מערך הנתונים ולהסיר את הערכים החריגים הללו, אבל זה יהפוך את הנתונים למינימליים מאוד.

בשלב זה, בואו נבחר אילו עמודות נשתמש בתרגיל האשכולות שלנו. נבחר את העמודות המספריות עם טווחים דומים. אפשר לקודד את `artist_top_genre` כערך מספרי, אבל כרגע נוותר עליו.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. חישוב אשכולות k-means ב-R

ניתן לחשב k-means ב-R באמצעות הפונקציה המובנית `kmeans`, ראו `help("kmeans()")`. הפונקציה `kmeans()` מקבלת מסגרת נתונים עם עמודות מספריות בלבד כארגומנט הראשי שלה.

השלב הראשון בשימוש באשכולות k-means הוא לקבוע את מספר האשכולות (k) שייווצרו בפתרון הסופי. אנחנו יודעים שיש 3 ז'אנרים של שירים שזיהינו מתוך מערך הנתונים, אז בואו ננסה 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


אובייקט ה-kmeans מכיל מספר פרטי מידע שמוסברים היטב ב-`help("kmeans()")`. כרגע, נתמקד בכמה מהם. אנו רואים שהנתונים חולקו ל-3 אשכולות בגודל 65, 110, 111. הפלט גם מכיל את מרכזי האשכולות (ממוצעים) עבור 3 הקבוצות על פני 5 המשתנים.

וקטור האשכולות הוא שיוך האשכול עבור כל תצפית. בואו נשתמש בפונקציה `augment` כדי להוסיף את שיוך האשכול לסט הנתונים המקורי.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


מעולה, חילקנו את מערך הנתונים שלנו ל-3 קבוצות. אז, עד כמה טובה החלוקה שלנו 🤷? בואו נבחן את `ציון הסילואט`.

### **ציון הסילואט**

[ניתוח סילואט](https://en.wikipedia.org/wiki/Silhouette_(clustering)) יכול לשמש כדי לבחון את מרחק ההפרדה בין הקבוצות שהתקבלו. ציון זה נע בין -1 ל-1, ואם הציון קרוב ל-1, הקבוצה צפופה ומופרדת היטב מקבוצות אחרות. ערך קרוב ל-0 מייצג קבוצות חופפות עם דגימות קרובות מאוד לגבול ההחלטה של הקבוצות השכנות. [מקור](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

שיטת הסילואט הממוצע מחשבת את ממוצע הסילואט של התצפיות עבור ערכים שונים של *k*. ציון סילואט ממוצע גבוה מעיד על חלוקה טובה.

הפונקציה `silhouette` בחבילת ה-cluster משמשת לחישוב רוחב הסילואט הממוצע.

> ניתן לחשב את הסילואט עם כל [מדד מרחק](https://en.wikipedia.org/wiki/Distance "Distance"), כמו [מרחק אוקלידי](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") או [מרחק מנהטן](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") שדנו בהם ב[שיעור הקודם](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


הציון שלנו הוא **.549**, כלומר ממש באמצע. זה מצביע על כך שהנתונים שלנו לא מתאימים במיוחד לסוג זה של אשכולות. בואו נראה אם נוכל לאשר את ההשערה הזו באופן חזותי. [חבילת factoextra](https://rpkgs.datanovia.com/factoextra/index.html) מספקת פונקציות (`fviz_cluster()`) כדי להמחיש אשכולות.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


החפיפה בין הקבוצות מצביעה על כך שהנתונים שלנו אינם מתאימים במיוחד לסוג זה של אשכולות, אך בואו נמשיך.

## 4. קביעת מספר האשכולות האופטימלי

שאלה בסיסית שעולה לעיתים קרובות באשכולות K-Means היא זו - ללא תוויות מחלקה ידועות, איך יודעים לכמה אשכולות יש לחלק את הנתונים?

אחת הדרכים לנסות לגלות זאת היא להשתמש בדגימת נתונים כדי `ליצור סדרה של מודלים אשכוליים` עם מספר אשכולות הולך וגדל (לדוגמה, מ-1 עד 10), ולהעריך מדדי אשכולות כמו **ציון הסילואט.**

בואו נקבע את מספר האשכולות האופטימלי על ידי חישוב האלגוריתם האשכולי עבור ערכים שונים של *k* והערכת **סכום הריבועים בתוך האשכולות** (WCSS). סכום הריבועים הכולל בתוך האשכולות (WCSS) מודד את הקומפקטיות של האשכולות, ואנו רוצים שהוא יהיה קטן ככל האפשר, כאשר ערכים נמוכים מצביעים על כך שנקודות הנתונים קרובות יותר.

בואו נחקור את ההשפעה של בחירות שונות של `k`, מ-1 עד 10, על האשכולות הללו.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


עכשיו, כשיש לנו את סכום הריבועים בתוך הקבוצה הכולל (tot.withinss) עבור כל אלגוריתם אשכולות עם מרכז *k*, אנו משתמשים בשיטת [המרפק](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) כדי למצוא את מספר האשכולות האופטימלי. השיטה כוללת שרטוט של WCSS כפונקציה של מספר האשכולות, ובחירת [המרפק של העקומה](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") כמספר האשכולות לשימוש.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


הגרף מציג ירידה משמעותית ב-WCSS (כלומר, *צפיפות* גבוהה יותר) כאשר מספר האשכולות עולה מאחד לשניים, וירידה נוספת ניכרת משניים לשלושה אשכולות. לאחר מכן, הירידה פחות בולטת, מה שיוצר "מרפק" 💪 בגרף בסביבות שלושה אשכולות. זהו סימן טוב לכך שישנם שניים עד שלושה אשכולות של נקודות נתונים המופרדים באופן סביר.

כעת נוכל להמשיך ולחלץ את מודל האשכולות שבו `k = 3`:

> `pull()`: משמש לחילוץ עמודה בודדת  
>  
> `pluck()`: משמש לאינדוקס מבני נתונים כמו רשימות  


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


מעולה! בואו נתקדם ונתבונן באשכולות שהתקבלו. מעוניינים להוסיף קצת אינטראקטיביות באמצעות `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


אולי היינו מצפים שלכל אשכול (המוצג בצבעים שונים) יהיו ז'אנרים מובחנים (המוצגים בצורות שונות).

בואו נבחן את דיוק המודל.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


הדיוק של המודל הזה לא רע, אבל גם לא מצוין. ייתכן שהנתונים אינם מתאימים היטב לקיבוץ באמצעות K-Means Clustering. הנתונים הללו אינם מאוזנים, הקורלציה ביניהם נמוכה מדי ויש יותר מדי שונות בין ערכי העמודות כדי ליצור קיבוץ מוצלח. למעשה, הקבוצות שנוצרות כנראה מושפעות או מוטות מאוד על ידי שלוש קטגוריות הז'אנר שהגדרנו קודם.

עם זאת, זו הייתה חוויה לימודית משמעותית!

בתיעוד של Scikit-learn, ניתן לראות שמודל כמו זה, שבו הקבוצות אינן מוגדרות היטב, סובל מבעיה של 'שונות':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>אינפוגרפיקה מתוך Scikit-learn</figcaption>



## **שונות**

שונות מוגדרת כ"הממוצע של ריבועי ההבדלים מהממוצע" [מקור](https://www.mathsisfun.com/data/standard-deviation.html). בהקשר של בעיית הקיבוץ הזו, היא מתייחסת לנתונים שבהם הערכים של מערך הנתונים שלנו נוטים לסטות יותר מדי מהממוצע.

✅ זהו רגע מצוין לחשוב על כל הדרכים שבהן ניתן לתקן את הבעיה הזו. לשנות מעט את הנתונים? להשתמש בעמודות אחרות? להשתמש באלגוריתם שונה? רמז: נסו [לשנות את קנה המידה של הנתונים](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) כדי לנרמל אותם ולבדוק עמודות אחרות.

> נסו את '[מחשבון השונות](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' כדי להבין את המושג קצת יותר.

------------------------------------------------------------------------

## **🚀אתגר**

הקדישו זמן למחברת הזו ושנו פרמטרים. האם תוכלו לשפר את דיוק המודל על ידי ניקוי הנתונים יותר (למשל, הסרת ערכים חריגים)? תוכלו להשתמש במשקלים כדי לתת משקל רב יותר לדגימות נתונים מסוימות. מה עוד תוכלו לעשות כדי ליצור קבוצות טובות יותר?

רמז: נסו לשנות את קנה המידה של הנתונים. יש קוד עם הערות במחברת שמוסיף שינוי קנה מידה סטנדרטי כדי לגרום לעמודות הנתונים להיות דומות יותר זו לזו מבחינת טווח. תגלו שבעוד שהציון של הסילואט יורד, ה'שבר' בגרף המרפק מתיישר. זאת משום שהשארת הנתונים ללא שינוי קנה מידה מאפשרת לנתונים עם פחות שונות לקבל משקל רב יותר. קראו עוד על הבעיה הזו [כאן](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**שאלון לאחר ההרצאה**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **סקירה ולימוד עצמי**

-   הסתכלו על סימולטור K-Means [כמו זה](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). תוכלו להשתמש בכלי הזה כדי להמחיש נקודות נתונים לדוגמה ולקבוע את המרכזים שלהן. תוכלו לערוך את רמת האקראיות של הנתונים, מספר הקבוצות ומספר המרכזים. האם זה עוזר לכם להבין כיצד ניתן לקבץ את הנתונים?

-   בנוסף, הסתכלו על [המסמך הזה על K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) מאוניברסיטת סטנפורד.

רוצים לנסות את כישורי הקיבוץ החדשים שלכם על מערכי נתונים שמתאימים היטב ל-K-Means Clustering? ראו:

-   [אימון והערכת מודלים של קיבוץ](https://rpubs.com/eR_ic/clustering) באמצעות Tidymodels וחברים

-   [ניתוח קיבוץ K-Means](https://uc-r.github.io/kmeans_clustering), מדריך לתכנות R של UC Business Analytics

- [קיבוץ K-Means עם עקרונות נתונים מסודרים](https://www.tidymodels.org/learn/statistics/k-means/)

## **משימה**

[נסו שיטות קיבוץ שונות](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## תודה ל:

[Jen Looper](https://www.twitter.com/jenlooper) על יצירת הגרסה המקורית של המודול הזה בפייתון ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) על יצירת האיורים המדהימים שהופכים את R למזמין ומרתק יותר. מצאו איורים נוספים בגלריה שלה [כאן](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

למידה נעימה,

[Eric](https://twitter.com/ericntay), שגריר סטודנטים של Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>יצירה מאת @allison_horst</figcaption>



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפתו המקורית נחשב למקור הסמכותי. למידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי מתרגם אנושי. איננו נושאים באחריות לכל אי-הבנה או פרשנות שגויה הנובעת משימוש בתרגום זה.  
